# Pandas Library

## Dataset creation

In [186]:
import pandas as pd

pd.DataFrame({
    'NumRoms': [None, 2.0, 4.0, None],
    'Alley': ['Pave', None, None, None],
    'Prices': [127500, 106000, 178100, 140000]
}).to_csv('data.csv', index = False)



In [187]:
dataset = pd.read_csv('data.csv')
dataset.head()

,NumRoms,Alley,Prices
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [188]:
s = dataset['Alley']
s

0    Pave
1     NaN
2     NaN
3     NaN
Name: Alley, dtype: object

In [189]:
s.loc[3]

nan

In [190]:
dataset.sample(3)

,NumRoms,Alley,Prices
2,4.0,NaN,178100
3,NaN,NaN,140000
1,2.0,NaN,106000


In [191]:
inputs, outputs = dataset.iloc[:, 0:2], dataset.iloc[:, 2]
#inputs.fillna(inputs.mean(), inplace = True, ) 
inputs.head()

,NumRoms,Alley
0,NaN,Pave
1,2.0,NaN
2,4.0,NaN
3,NaN,NaN


In [192]:
#isolate numercial columns and object columns:
num_columns = inputs.select_dtypes(include= 'float64')


#update numercial columns with fillna using mean():
inputs[num_columns.columns.to_list()] = num_columns.fillna(num_columns.mean())


#results:
inputs


,NumRoms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [193]:
inputs = pd.get_dummies(inputs , dummy_na = True) + 0
inputs 

,NumRoms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


In [194]:
import torch as tr

x, y = tr.tensor(inputs.values), tr.tensor(outputs.values)

x, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## Exercise

#### raw dataset generation

In [195]:
#unique values of each column:
brands = ['HP', 'DELL', 'LENOVO', 'ASUS', 'ACER', 'TOSHIBA', None]
processors = ['i3', 'i5', 'i7', 'i9', 'Rayzen', None]
generations = [i for i in range(3, 13)] + [None]
rams = [2 ** i for i in range(1, 6)] + [None]
stockages = [2 ** i for i in range(6, 11)] + [None]
prices =  [p for p in range(1000, 8000, 500)]


#generate a random indexs matrix:
import random as rnd

rnd.seed(100)
def gen_matrix(size: int, *dims: int) -> list[list[int]]:
    return [[rnd.randint(0, b - 1) for b in dims] for _ in range(size)]


tp = (brands, processors, generations, rams, stockages, prices)

#generate data based on random indexs matrix:
data = list(
    map(
        lambda lst: list(
            map(
                lambda it: tp[it[0]][it[1]],
                enumerate(lst)
            )
        ),
        gen_matrix(
            150, 
            len(brands), 
            len(processors), 
            len(generations), 
            len(rams), 
            len(stockages), 
            len(prices)
        )
    )
)


#create a dataframe:

laptops = pd.DataFrame(
    data = data,
    columns = ['brand', 'processor', 'gen', 'RAM', 'HD', 'price'],
)

laptops



,brand,processor,gen,RAM,HD,price
0,DELL,i9,10.0,4.0,NaN,4000
1,TOSHIBA,i7,9.0,32.0,64.0,5000
2,HP,i3,10.0,8.0,64.0,6000
3,TOSHIBA,i5,8.0,4.0,256.0,7500
4,None,i5,5.0,4.0,128.0,3500
...,...,...,...,...,...,...
145,None,i7,10.0,2.0,64.0,7500
146,TOSHIBA,i9,10.0,32.0,NaN,7500
147,LENOVO,None,6.0,16.0,64.0,5000
148,ASUS,Rayzen,3.0,2.0,256.0,2000


#### remove column with highest nan values

In [196]:
#get number of nan values for each column:
boolean_ = laptops.isna()
nan_by_columns = dict(boolean_.apply(func = lambda col: col.values.sum()))
nan_by_columns

{'brand': 17, 'processor': 31, 'gen': 8, 'RAM': 27, 'HD': 30, 'price': 0}

In [197]:
#drop columns with highest number of nan values:
laptops.drop(max(nan_by_columns), axis = 1, inplace = True)
laptops

,brand,gen,RAM,HD,price
0,DELL,10.0,4.0,NaN,4000
1,TOSHIBA,9.0,32.0,64.0,5000
2,HP,10.0,8.0,64.0,6000
3,TOSHIBA,8.0,4.0,256.0,7500
4,None,5.0,4.0,128.0,3500
...,...,...,...,...,...
145,None,10.0,2.0,64.0,7500
146,TOSHIBA,10.0,32.0,NaN,7500
147,LENOVO,6.0,16.0,64.0,5000
148,ASUS,3.0,2.0,256.0,2000


#### separate features and target

In [198]:
#separate features and target
features, target = laptops.iloc[:, 0:4], laptops.iloc[:, 4]
features

,brand,gen,RAM,HD
0,DELL,10.0,4.0,NaN
1,TOSHIBA,9.0,32.0,64.0
2,HP,10.0,8.0,64.0
3,TOSHIBA,8.0,4.0,256.0
4,None,5.0,4.0,128.0
...,...,...,...,...
145,None,10.0,2.0,64.0
146,TOSHIBA,10.0,32.0,NaN
147,LENOVO,6.0,16.0,64.0
148,ASUS,3.0,2.0,256.0


In [199]:
target

0      4000
1      5000
2      6000
3      7500
4      3500
       ... 
145    7500
146    7500
147    5000
148    2000
149    2000
Name: price, Length: 150, dtype: int64

#### categorizing based on brand

In [200]:
#categorizing based on brand 
features =  pd.get_dummies(features) + 0
features

,gen,RAM,HD,brand_ACER,brand_ASUS,brand_DELL,brand_HP,brand_LENOVO,brand_TOSHIBA
0,10.0,4.0,NaN,0,0,1,0,0,0
1,9.0,32.0,64.0,0,0,0,0,0,1
2,10.0,8.0,64.0,0,0,0,1,0,0
3,8.0,4.0,256.0,0,0,0,0,0,1
4,5.0,4.0,128.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
145,10.0,2.0,64.0,0,0,0,0,0,0
146,10.0,32.0,NaN,0,0,0,0,0,1
147,6.0,16.0,64.0,0,0,0,0,1,0
148,3.0,2.0,256.0,0,1,0,0,0,0


#### fill nan values

In [201]:
#replace numerical nan values:
features.fillna(method = 'ffill' , inplace = True)
features.fillna(method = 'bfill' , inplace = True)
features

,gen,RAM,HD,brand_ACER,brand_ASUS,brand_DELL,brand_HP,brand_LENOVO,brand_TOSHIBA
0,10.0,4.0,64.0,0,0,1,0,0,0
1,9.0,32.0,64.0,0,0,0,0,0,1
2,10.0,8.0,64.0,0,0,0,1,0,0
3,8.0,4.0,256.0,0,0,0,0,0,1
4,5.0,4.0,128.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
145,10.0,2.0,64.0,0,0,0,0,0,0
146,10.0,32.0,64.0,0,0,0,0,0,1
147,6.0,16.0,64.0,0,0,0,0,1,0
148,3.0,2.0,256.0,0,1,0,0,0,0


#### create tensor format from featues and target

In [202]:
#get tensor format from features and target:
xies, y =  tr.tensor(features.values), tr.tensor(target.values)

xies, y

(tensor([[ 10.,   4.,  64.,  ...,   0.,   0.,   0.],
         [  9.,  32.,  64.,  ...,   0.,   0.,   1.],
         [ 10.,   8.,  64.,  ...,   1.,   0.,   0.],
         ...,
         [  6.,  16.,  64.,  ...,   0.,   1.,   0.],
         [  3.,   2., 256.,  ...,   0.,   0.,   0.],
         [  3.,   4., 512.,  ...,   0.,   0.,   1.]], dtype=torch.float64),
 tensor([4000, 5000, 6000, 7500, 3500, 4500, 6000, 2000, 7500, 4000, 2500, 6500,
         5000, 7500, 2500, 5500, 7500, 6500, 3000, 6500, 6500, 6500, 1500, 6000,
         2000, 2000, 1500, 4000, 3500, 5500, 7500, 7500, 1000, 1000, 1000, 6000,
         3500, 7000, 7500, 1000, 5000, 4500, 5000, 6500, 1500, 5500, 6500, 4000,
         6500, 5000, 2000, 6500, 6000, 7000, 2500, 4500, 4500, 5500, 7500, 6500,
         3500, 3000, 3500, 5500, 3500, 6000, 2000, 7500, 4500, 4000, 3000, 6500,
         6000, 7000, 5000, 1000, 3500, 5500, 3500, 3500, 7000, 2000, 7000, 4000,
         3500, 7500, 2500, 1500, 4000, 5000, 3000, 6000, 5500, 4000, 7500, 450